In [1]:
#!pip install -q --upgrade nltk gensim bokeh pandas

In [2]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\rost_\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rost_\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
from sklearn.manifold import TSNE

C:\Users\rost_\Anaconda3\envs\nlp\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [4]:
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from string import punctuation

## Запилим пословный машинный перевод!

In [5]:
#!wget -O ukr_rus.train.txt -qq --no-check-certificate "https://drive.google.com/uc?export=download&id=1vAK0SWXUqei4zTimMvIhH3ufGPsbnC_O"
#!wget -O ukr_rus.test.txt -qq --no-check-certificate "https://drive.google.com/uc?export=download&id=1W9R2F8OeKHXruo2sicZ6FgBJUTJc8Us_"
#!wget -O fairy_tale.txt -qq --no-check-certificate "https://drive.google.com/uc?export=download&id=1sq8zSroFeg_afw-60OmY8RATdu_T1tej"

# Install the PyDrive wrapper & import libraries.
# This only needs to be done once per notebook.
#!pip install -U -q PyDrive

Напишем простенькую реализацию модели машинного перевода.

Идея основана на статье [Word Translation Without Parallel Data](https://arxiv.org/pdf/1710.04087.pdf). У авторов в репозитории еще много интересного: [https://github.com/facebookresearch/MUSE](https://github.com/facebookresearch/MUSE).

А мы будем переводить с украинского на русский.

![](https://raw.githubusercontent.com/yandexdataschool/nlp_course/master/resources/blue_cat_blue_whale.png)   
*синій кіт* vs. *синій кит*

In [6]:
from gensim.models import KeyedVectors

ru_emb = KeyedVectors.load_word2vec_format("./cc.ru.300.vec/cc.ru.300.vec")
uk_emb = KeyedVectors.load_word2vec_format("./cc.uk.300.vec/cc.uk.300.vec")

Посмотрим на пару серпень-август (являющихся переводом)

In [172]:
ru_emb.most_similar([ru_emb["август"]])

[('август', 1.0),
 ('июль', 0.9383152723312378),
 ('сентябрь', 0.9240028262138367),
 ('июнь', 0.9222574830055237),
 ('октябрь', 0.9095539450645447),
 ('ноябрь', 0.893003523349762),
 ('апрель', 0.8729087114334106),
 ('декабрь', 0.8652557730674744),
 ('март', 0.8545796275138855),
 ('февраль', 0.8401415944099426)]

In [173]:
uk_emb.most_similar([uk_emb["серпень"]])

[('серпень', 1.0),
 ('липень', 0.9096439480781555),
 ('вересень', 0.9016969203948975),
 ('червень', 0.8992519974708557),
 ('жовтень', 0.8810407519340515),
 ('листопад', 0.8787633776664734),
 ('квітень', 0.8592804074287415),
 ('грудень', 0.8586863279342651),
 ('травень', 0.840811014175415),
 ('лютий', 0.8256431221961975)]

In [174]:
ru_emb.most_similar([uk_emb["серпень"]])

[('Stepashka.com', 0.2757962644100189),
 ('ЖИЗНИВадим', 0.25203433632850647),
 ('2Дмитрий', 0.25048115849494934),
 ('2012Дмитрий', 0.24829229712486267),
 ('Ведущий-Алексей', 0.2443869411945343),
 ('Недопустимость', 0.24435284733772278),
 ('2Михаил', 0.23981401324272156),
 ('лексей', 0.23740755021572113),
 ('комплексн', 0.23695150017738342),
 ('персональ', 0.2368222326040268)]

In [175]:
def load_word_pairs(filename):
    uk_ru_pairs = []
    uk_vectors = []
    ru_vectors = []
    with open(filename, "r", encoding='utf8') as inpf:
        for line in inpf:
            uk, ru = line.rstrip().split("\t")
            if uk not in uk_emb or ru not in ru_emb:
                continue
            uk_ru_pairs.append((uk, ru))
            uk_vectors.append(uk_emb[uk])
            ru_vectors.append(ru_emb[ru])
    return uk_ru_pairs, np.array(uk_vectors), np.array(ru_vectors)


uk_ru_train, X_train, Y_train = load_word_pairs("ukr_rus.train.txt")
uk_ru_test, X_test, Y_test = load_word_pairs("ukr_rus.test.txt")

### Учим маппинг из одного пространства эмбеддингов в другое

У нас есть пары слов, соответствующих друг другу, и их эмбеддинги. Найдем преобразование из одного пространства в другое, чтобы приблизить известные нам слова:

$$W^*= \arg\min_W ||WX - Y||_F, \text{где} ||*||_F - \text{норма Фробениуса}$$

Эта функция очень похожа на линейную регрессию (без биаса).

**Задание** Реализуйте её - воспользуйтесь `LinearRegression` из sklearn с `fit_intercept=False`:

In [176]:
import numpy as np
from sklearn.linear_model import LinearRegression

mapping = LinearRegression(fit_intercept=False).fit(X_train, Y_train)
mapping.score(X_train, Y_train)

0.5359725797652927

In [177]:
y_hat = mapping.predict(X_test)

from sklearn.metrics import mean_absolute_error, mean_squared_error
mean_absolute_error(Y_test, y_hat), mean_squared_error(Y_test, y_hat)

(0.033866413, 0.0025563438)

Проверим, куда перейдет `серпень`:

In [178]:
august = mapping.predict(uk_emb["серпень"].reshape(1, -1))
ru_emb.most_similar(august)

[('апрель', 0.8531433343887329),
 ('июнь', 0.8402523398399353),
 ('март', 0.8385883569717407),
 ('сентябрь', 0.8331484794616699),
 ('февраль', 0.8311208486557007),
 ('октябрь', 0.8278018832206726),
 ('ноябрь', 0.8243728876113892),
 ('июль', 0.822961688041687),
 ('август', 0.8112280368804932),
 ('январь', 0.8022985458374023)]

Должно получиться, что в топе содержатся разные месяцы, но август не первый.

Будем мерять percision top-k с k = 1, 5, 10.

**Задание** Реализуйте следующую функцию:

In [179]:
def precision(pairs, mapped_vectors, topn=1):
    """
    :args:
        pairs = list of right word pairs [(uk_word_0, ru_word_0), ...]
        mapped_vectors = list of embeddings after mapping from source embedding space to destination embedding space
        topn = the number of nearest neighbours in destination embedding space to choose from
    :returns:
        precision_val, float number, total number of words for those we can find right translation at top K.
    """
    assert len(pairs) == len(mapped_vectors)
    num_matches = 0
    for i, (_, ru) in enumerate(pairs):
        for j in range(topn):
            if pairs[i][1] in ru_emb.most_similar(mapped_vectors[i].reshape(1, -1), topn=topn)[j][0]:
                num_matches = num_matches+1
                break
        
        
    precision_val = num_matches / len(pairs)
    return precision_val

In [180]:
precision(uk_ru_test, y_hat, topn=10)

0.8549618320610687

In [181]:
assert precision([("серпень", "август")], august, topn=5) == 0.0
assert precision([("серпень", "август")], august, topn=9) == 1.0
assert precision([("серпень", "август")], august, topn=10) == 1.0

In [182]:
assert precision(uk_ru_test, X_test) == 0.0
assert precision(uk_ru_test, Y_test) == 1.0

In [183]:
precision_top1 = precision(uk_ru_test, mapping.predict(X_test), 1)
precision_top10 = precision(uk_ru_test, mapping.predict(X_test), 10)

assert precision_top1 >= 0.635
assert precision_top10 >= 0.813

### Улучшаем маппинг

Можно показать, что маппинг лучше строить ортогональным:
$$W^*= \arg\min_W ||WX - Y||_F \text{, где: } W^TW = I$$

Искать его можно через SVD:
$$X^TY=U\Sigma V^T\text{, singular value decompostion}$$

$$W^*=UV^T$$

**Задание** Реализуйте эту функцию.

In [184]:
def learn_transform(X_train, Y_train):
    from numpy import linalg as LA
    """ 
    :returns: W* : float matrix[emb_dim x emb_dim] as defined in formulae above
    """
    XTY=np.matmul(X_train.T, Y_train)
    u, s, vh = LA.svd(XTY, full_matrices=False)
    W_=np.matmul(u, vh)
    
    return W_
    

In [185]:
W = learn_transform(X_train, Y_train)

In [186]:
ru_emb.most_similar([np.matmul(uk_emb["серпень"], W)])

[('апрель', 0.8245132565498352),
 ('июнь', 0.8056630492210388),
 ('сентябрь', 0.8055762648582458),
 ('март', 0.8032936453819275),
 ('октябрь', 0.7987101674079895),
 ('июль', 0.7946798205375671),
 ('ноябрь', 0.793963611125946),
 ('август', 0.793819010257721),
 ('февраль', 0.7923861742019653),
 ('декабрь', 0.7715376019477844)]

In [187]:
y_hat = np.matmul(X_test, W)
precision(uk_ru_test, y_hat, topn=10)

0.8498727735368957

In [188]:
assert precision(uk_ru_test, np.matmul(X_test, W)) >= 0.653
assert precision(uk_ru_test, np.matmul(X_test, W), 10) >= 0.824

### Пишем переводчик

Реализуем простой пословный переводчик - для каждого слова будем искать его ближайшего соседа в общем пространстве эмбеддингов. Если слова нет в эмбеддингах - просто копируем его.

In [189]:
with open("fairy_tale.txt", "r",encoding='UTF-8') as f:
    uk_sentences = [line.rstrip().lower() for line in f]

In [190]:
def translate(sentence):
    """
    :args:
        sentence - sentence in Ukrainian (str)
    :returns:
        translation - sentence in Russian (str)

    * find ukrainian embedding for each word in sentence
    * transform ukrainian embedding vector
    * find nearest russian word and replace
    """
    def words_only(text):
        return text.lower()   
    sentence = words_only(sentence)
    
    def remove_punkt(text, ptrn):
        return re.sub(ptrn, ' ', text)

    sentence = remove_punkt(sentence, r'[^\w\s]') 
        
    def to_token(text):
        return nltk.tokenize.word_tokenize(text)

    sentence = to_token(sentence)         
    
    result=''
    for word in sentence:    
        try:
            tr_word = np.matmul(uk_emb[word], W)
            result = result + ru_emb.most_similar(tr_word.reshape(1,-1), topn=1)[0][0] + ' '        
        except:
            result = result + word + ' '        
    
    return result
    


In [193]:
#assert translate(".") == "."
#assert translate("1 , 3") == "1 , 3"
#assert translate("кіт зловив мишу") == "кот поймал мышку"

In [194]:
translate("кіт зловив мишу")

'кот поймал мышку '